In [1]:
import pandas as pd
import os, os.path
import sqlite3
import tempfile


In [2]:
dataframes = [
    "Badges",
    "Comments",
    "PostLinks",
    "Posts",
    "Tags",
    "Users",
    "Votes",
]

for dataframe in dataframes:
    globals()[dataframe] = pd.read_csv(
        os.path.join("data", f"{dataframe}.csv.gz"), compression="gzip"
    )


In [3]:
# sciezka dostępu do bazy danych:
baza = os.path.join(tempfile.mkdtemp(), "przyklad.db")
if os.path.isfile(baza):  # jesli baza już istneje...
    os.remove(baza)  # ...usuniemy ja
conn = sqlite3.connect(baza)  # połączenie do bazy danych
Badges.to_sql("Badges", conn)  # importujemy ramkę danych do bazy danych
Comments.to_sql("Comments", conn)
PostLinks.to_sql("PostLinks", conn)
Posts.to_sql("Posts", conn)
Tags.to_sql("Tags", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)


# Zadania

## zapytanie 1

In [4]:
sql_output = pd.read_sql_query(
    """
SELECT Location, COUNT(*) AS Count
FROM (
    SELECT Posts.OwnerUserId, Users.Id, Users.Location
    FROM Users
    JOIN Posts ON Users.Id = Posts.OwnerUserId
)
WHERE Location NOT IN ('')
GROUP BY Location
ORDER BY Count DESC
LIMIT 10
""",
    conn,
)
sql_output
## W powyższym zapytaniu chcemy znaleźć 10 krajów, z których najczęście wstawiano posty


,Location,Count
0,"Christchurch, New Zealand",2765
1,"New York, NY",1788
2,"London, United Kingdom",1708
3,UK,1590
4,"Sunshine Coast QLD, Australia",1550
5,Australia,1183
6,"Vancouver, Canada",967
7,Netherlands,935
8,on the server farm,924
9,Pennsylvania,921


In [5]:
# Na początku dokonujemy JOIN'a aby mieć tabele z postami i lokalizacjami
# osób które je udostępniły
temp_df = pd.merge(
    Posts[["OwnerUserId"]],
    Users[["Id", "Location"]],
    left_on="OwnerUserId",
    right_on="Id",
)
# wyrzucamy recordy z pustą lokacją
temp_df = temp_df[temp_df["Location"] != ""]
# dokonujemy grupowania ze względu na lokację i zliczamy ile postów było z tej lokacji wstawionych
temp_df = temp_df.groupby(["Location"]).count()[["OwnerUserId"]].reset_index()
# zmieniamy nazwy kolumnu aby były zgodne z tymi w zapytaniu
temp_df.columns = ["Location", "Count"]
# sortujemy po liczbie postów i zostawiamy 10 rekordów z największymi liczbami postów
temp_df = temp_df.sort_values(by="Count", ascending=False).reset_index(
    drop=True
)
temp_df = temp_df.head(10)

temp_df


,Location,Count
0,"Christchurch, New Zealand",2765
1,"New York, NY",1788
2,"London, United Kingdom",1708
3,UK,1590
4,"Sunshine Coast QLD, Australia",1550
5,Australia,1183
6,"Vancouver, Canada",967
7,Netherlands,935
8,on the server farm,924
9,Pennsylvania,921


In [6]:
temp_df.equals(sql_output)
## Taki sam wynik jak dla zapytania sql


True

## zapytanie 2

In [7]:
sql_output = pd.read_sql_query(
    """
SELECT
    Users.AccountId,
    Users.DisplayName,
    Users.Location,
    AVG(PostAuth.AnswersCount) as AverageAnswersCount
FROM
(
    SELECT
        AnsCount.AnswersCount,
        Posts.Id,
        Posts.OwnerUserId
    FROM (
        SELECT Posts.ParentId, COUNT(*) AS AnswersCount
        FROM Posts
        WHERE Posts.PostTypeId = 2
        GROUP BY Posts.ParentId
        ) AS AnsCount
    JOIN Posts ON Posts.Id = AnsCount.ParentId
) AS PostAuth
JOIN Users ON Users.AccountId=PostAuth.OwnerUserId
GROUP BY OwnerUserId
ORDER BY AverageAnswersCount DESC
LIMIT 10
""",
    conn,
)
sql_output
# w powyższym zapytaniu szukamy użytkowników, którzy mają największą liczbę odpowiedzi pod swoimi postami typu 2


,AccountId,DisplayName,Location,AverageAnswersCount
0,40811.0,vocaro,"San Jose, CA",11.0
1,280.0,csmba,"San Francisco, CA",11.0
2,44093.0,Emma Arbogast,"Salem, OR",10.0
3,204.0,Josh,Australia,10.0
4,11758.0,rvarcher,"Oklahoma City, OK",9.0
5,79346.0,Thomas Matthews,California,8.0
6,54571.0,Christian,None,8.0
7,42364.0,Petrogad,None,8.0
8,20473.0,Jeremy Boyd,"Houston, TX",8.0
9,19588.0,JD Isaacks,"Atlanta, GA",8.0


In [8]:
# Tworzymy tabelę z liczbami odpowiedzi
AnsCount = Posts[Posts["PostTypeId"] == 2]
AnsCount = AnsCount.groupby("ParentId").count()
AnsCount = AnsCount[["Id"]]
AnsCount.columns = ["AnswersCount"]
AnsCount = AnsCount.reset_index()
# JOINujemy tabelę z liczbami odpowiedzi z tabelą postów
PostAuth = pd.merge(
    AnsCount, Posts[["Id", "OwnerUserId"]], left_on="ParentId", right_on="Id"
)
# Obliczamy średnią liczbę odpowiedzi dla postów użytkowników
temp_df = PostAuth.groupby("OwnerUserId").mean()[["AnswersCount"]]
temp_df.columns = ["AverageAnswersCount"]
temp_df = temp_df.reset_index()
## JOINujemy tabelę użytkowników z tabelą ze średnią liczbą odpowiedzi pod postami
temp_df = pd.merge(
    Users[["AccountId", "DisplayName", "Location"]],
    temp_df,
    left_on="AccountId",
    right_on="OwnerUserId",
)
# Dokonujemy sortowania względem liczby średniej odpowiedzi i wybieramy 10 pierwszych rekordów
temp_df = (
    temp_df.sort_values(by="AverageAnswersCount", ascending=False)
    .reset_index(drop=True)
    .head(10)
)
temp_df


,AccountId,DisplayName,Location,OwnerUserId,AverageAnswersCount
0,40811.0,vocaro,"San Jose, CA",40811.0,11.0
1,280.0,csmba,"San Francisco, CA",280.0,11.0
2,204.0,Josh,Australia,204.0,10.0
3,44093.0,Emma Arbogast,"Salem, OR",44093.0,10.0
4,11758.0,rvarcher,"Oklahoma City, OK",11758.0,9.0
5,42364.0,Petrogad,NaN,42364.0,8.0
6,19588.0,JD Isaacks,"Atlanta, GA",19588.0,8.0
7,54571.0,Christian,NaN,54571.0,8.0
8,20473.0,Jeremy Boyd,"Houston, TX",20473.0,8.0
9,79346.0,Thomas Matthews,California,79346.0,8.0


In [9]:
temp_df.merge(sql_output, indicator=True, how="outer")["_merge"].eq(
    "both"
).all()
## Taki sam wynik jak dla zapytania sql


True

## Zapytanie 3

In [10]:
sql_output = pd.read_sql_query(
    """
SELECT
    Posts.Title,
    UpVotesPerYear.Year,
MAX(UpVotesPerYear.Count) AS Count
FROM (
    SELECT
        PostId,
        COUNT(*) AS Count,
        STRFTIME('%Y', Votes.CreationDate) AS Year
    FROM Votes
    WHERE VoteTypeId=2
    GROUP BY PostId, Year
    ) AS UpVotesPerYear
JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
WHERE Posts.PostTypeId=1
GROUP BY Year
ORDER BY Year ASC
""",
    conn,
)
sql_output
# W powyższym zapytaniu szukamy tytułów postów, które w danym roku
# miały największą liczbę polubień 

,Title,Year,Count
0,"OK we're all adults here, so really, how on ea...",2011,70
1,How to successfully haggle / bargain in markets,2012,37
2,Why are airline passengers asked to lift up wi...,2013,103
3,How do you know if Americans genuinely/literal...,2014,179
4,Immigration officer that stopped me at the air...,2015,117
5,I don't know my nationality. How can I visit D...,2016,134
6,Why prohibit engine braking?,2017,177
7,How can I find restaurants in the USA where ti...,2018,119
8,My name causes an issue with any booking! (nam...,2019,263
9,What's the longest distance that can be travel...,2020,110


In [11]:
# zamieniamy kolumnę na typ DateTime
Votes["CreationDate"] = pd.to_datetime(Votes["CreationDate"])


In [12]:
# Wybieramy polubienia o typie 2.
UpVotesPerYear = Votes[Votes["VoteTypeId"] == 2]
# Dodajemy kolumnę odpowiadającą roku, w którym
# było dodane polubienie
UpVotesPerYear["Year"] = Votes["CreationDate"].apply(lambda x: str(x.year))
# Zliczamy liczbę polubień ze względu na rok
UpVotesPerYear = UpVotesPerYear.groupby(["PostId", "Year"]).count()[["Id"]]
UpVotesPerYear.columns = ["Count"]
UpVotesPerYear = UpVotesPerYear.reset_index()
# JOINujemy z tabelą Posts w celu posiadania informacji o postach
temp_df = pd.merge(Posts, UpVotesPerYear, left_on="Id", right_on="PostId")
# Wybieramy posty typu 1.
temp_df = temp_df[temp_df["PostTypeId"] == 1]
# Znajdujemy indeksy dla, których w danym roku było najwięcej polubień
max_idx = temp_df.groupby("Year").idxmax()
# wybieramy wcześniej znalezione indeksy i dokonujemy 
# posortowania tabeli ze względu na rok
temp_df = temp_df.loc[
    max_idx["Count"], ["Title", "Year", "Count"]
].reset_index(drop=True)
temp_df = temp_df.sort_values(by="Year")
temp_df


/tmp/ipykernel_136665/1139819639.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UpVotesPerYear["Year"] = Votes["CreationDate"].apply(lambda x: str(x.year))
/home/mchraba/anaconda3/envs/padpy/lib/python3.8/site-packages/pandas/core/groupby/groupby.py:1309: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  keys, values, mutated = self.grouper.apply(f, data, self.axis)


,Title,Year,Count
0,"OK we're all adults here, so really, how on ea...",2011,70
1,How to successfully haggle / bargain in markets,2012,37
2,Why are airline passengers asked to lift up wi...,2013,103
3,How do you know if Americans genuinely/literal...,2014,179
4,Immigration officer that stopped me at the air...,2015,117
5,I don't know my nationality. How can I visit D...,2016,134
6,Why prohibit engine braking?,2017,177
7,How can I find restaurants in the USA where ti...,2018,119
8,My name causes an issue with any booking! (nam...,2019,263
9,What's the longest distance that can be travel...,2020,110


In [13]:
temp_df.merge(sql_output, indicator=True, how="outer")["_merge"].eq(
    "both"
).all()
## Taki sam wynik jak dla zapytania sql


True